In [1]:
import pandas as pd
import json

with open('DICCIONARIO_CODIGO_NOMBRE_FARMACOS.json', 'r', encoding = 'utf-8') as f:
    DICCIONARIO_CODIGO_NOMBRE_FARMACOS = json.load(f)

In [2]:
df = pd.read_excel('asd_data.xls')

In [3]:
class Formateador():
    def __init__(self):
        pass

    def obtener_entradas_todos_los_pacientes(self):
        entradas_todos_los_pacientes = []

        for nombre_archivo in os.listdir():
            if '.xls' in nombre_archivo:
                entradas_de_un_paciente = self.obtener_entradas_de_un_paciente(nombre_archivo)
                entradas_todos_los_pacientes += entradas_de_un_paciente

        return entradas_todos_los_pacientes

    def obtener_entradas_de_un_paciente(self, nombre_archivo):
        entradas = []

        datos_persona = self.obtener_datos_demograficos_de_un_paciente(nombre_archivo)
        lista_antibiogramas_persona = self.obtener_antibiogramas_de_un_paciente(nombre_archivo)
        lista_contaminados_persona = self.obtener_contaminados_de_un_paciente(nombre_archivo)
        lista_hongos_persona = self.obtener_cultivos_de_hongos_de_un_paciente(nombre_archivo)

        for antibiograma in lista_antibiogramas_persona:
            entrada_antibiograma = datos_persona + antibiograma
            entradas.append(entrada_antibiograma)
        
        for contaminados in lista_contaminados_persona:
            entrada_contaminados = datos_persona + contaminados
            entradas.append(entrada_contaminados)
        
        for hongos in lista_hongos_persona:
            entrada_hongos = datos_persona + hongos
            entradas.append(entrada_hongos)


    
    def obtener_antibiogramas_de_un_paciente(self, nombre_archivo):
        pass

    def obtener_contaminados_de_un_paciente(self, nombre_archivo):
        pass

    def obtener_cultivos_de_hongos_de_un_paciente(self, nombre_archivo):
        pass

    def obtener_datos_demograficos_de_un_paciente(self, nombre_archivo):
        pass




In [3]:
def separar_tablas_por_cepa(tabla_total_antibiograma):
    headers = ['ANTIBIOTICO']
    cantidad_de_cepas = int((len(tabla_total_antibiograma.columns) - 1)/ 2)
    for i in range(cantidad_de_cepas):
        headers.append(f'Cepa')
        headers.append(f'CIM')

    tabla_total_antibiograma.columns = headers
    tabla_total_antibiograma.ANTIBIOTICO = tabla_total_antibiograma.ANTIBIOTICO.map(DICCIONARIO_CODIGO_NOMBRE_FARMACOS)
    tabla_total_antibiograma.set_index(tabla_total_antibiograma.ANTIBIOTICO, inplace = True)
    tabla_total_antibiograma.drop(columns = 'ANTIBIOTICO', inplace = True)

    tablas_cepas = []
    for i in range(cantidad_de_cepas):
        df = tabla_total_antibiograma.iloc[:, i * 2: (i * 2) + 2]
        df = df[df['Cepa'].notna()]
        tablas_cepas.append(df)
    
    tablas_cepas = list(map(mappear_resultados_a_formato_excel, tablas_cepas))
    
    return tablas_cepas

In [4]:
def mappear_resultados_a_formato_excel(tabla_una_cepa):
    diccionario_sensibilidades_a_llenar = {farmaco: None for farmaco in DICCIONARIO_CODIGO_NOMBRE_FARMACOS.values()}
    for farmaco in tabla_una_cepa.index:
        resultado_sensibilidad = tabla_una_cepa['Cepa'][farmaco]
        diccionario_sensibilidades_a_llenar[farmaco] = resultado_sensibilidad
    
    lista_sensibilidades_llenas = list(diccionario_sensibilidades_a_llenar.values())
    return lista_sensibilidades_llenas
    

In [17]:
def obtener_datos_un_paciente(self, nombre_archivo_paciente):
    #lista_datos_personales = self.obtener_datos_personales_paciente(nombre_archivo_paciente)
    lista_resultados_sensibilidad = self.obtener_tabla_antibiograma(nombre_archivo_paciente)

    registro_un_paciente = []
    for resultado in lista_resultados_sensibilidad:
        #registro_un_paciente.append(lista_datos_personales + resultado)
        registro_un_paciente.append(['A', 'B', 'C'] + resultado)

    return registro_un_paciente


In [6]:
tabla_total_antibiograma = df.iloc[18:31, :-2]
tablas_cepas = separar_tablas_por_cepa(tabla_total_antibiograma)
df = obtener_datos_un_paciente(tablas_cepas)

In [9]:
tablas_cepas

[[None,
  None,
  None,
  None,
  None,
  None,
  'Resistente',
  None,
  None,
  None,
  'Sensible',
  'Sensible',
  None,
  'Sensible',
  'Resistente',
  None,
  'Sensible',
  None,
  None,
  None,
  None,
  None,
  None,
  'Sensible',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  'Sensible'],
 ['Resistente',
  None,
  None,
  None,
  'Sensible',
  None,
  None,
  'Resistente',
  None,
  None,
  'Resistente',
  None,
  None,
  'Sensible',
  None,
  None,
  'Resistente',
  None,
  'Sensible',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  'Sensible',
  None,
  None,
  None,
  None,
  None,
  None,
  None]]

In [15]:
asd = ['a', 'b', 'c']
registro_un_paciente = []
for resultado in tablas_cepas:
    registro_un_paciente.append(asd + resultado)

registro_un_paciente

[['a',
  'b',
  'c',
  None,
  None,
  None,
  None,
  None,
  None,
  'Resistente',
  None,
  None,
  None,
  'Sensible',
  'Sensible',
  None,
  'Sensible',
  'Resistente',
  None,
  'Sensible',
  None,
  None,
  None,
  None,
  None,
  None,
  'Sensible',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  'Sensible'],
 ['a',
  'b',
  'c',
  'Resistente',
  None,
  None,
  None,
  'Sensible',
  None,
  None,
  'Resistente',
  None,
  None,
  'Resistente',
  None,
  None,
  'Sensible',
  None,
  None,
  'Resistente',
  None,
  'Sensible',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  'Sensible',
  None,
  None,
  None,
  None,
  None,
  None,
  None]]

In [16]:
registro_un_paciente

[['a',
  'b',
  'c',
  None,
  None,
  None,
  None,
  None,
  None,
  'Resistente',
  None,
  None,
  None,
  'Sensible',
  'Sensible',
  None,
  'Sensible',
  'Resistente',
  None,
  'Sensible',
  None,
  None,
  None,
  None,
  None,
  None,
  'Sensible',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  'Sensible'],
 ['a',
  'b',
  'c',
  'Resistente',
  None,
  None,
  None,
  'Sensible',
  None,
  None,
  'Resistente',
  None,
  None,
  'Resistente',
  None,
  None,
  'Sensible',
  None,
  None,
  'Resistente',
  None,
  'Sensible',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  'Sensible',
  None,
  None,
  None,
  None,
  None,
  None,
  None]]